# Load and wrangle dataset

In [8]:
import pandas as pd, numpy as np

df = pd.read_csv("data/processed/phq4_ts_features.csv")

# keep rows that have SRI and at least 7 PHQ-4 observations
df = df.dropna(subset=["SRI"])
valid_pids = df["pid"].value_counts().loc[lambda s: s >= 7].index
df = df[df["pid"].isin(valid_pids)]

# drop very sparse columns (>1000 missing) & keep predictors only
missing = df.isna().sum()
keep_cols = missing[missing < 1000].index.difference(
    ["pid", "PHQ4_date", "PHQ4"]
)
# --- after you load & basic-filter df ---------------------------
df = df.sort_values(["pid", "PHQ4_date"])

# 1. create deviation-from-subject-mean
df["PHQ4_dev"] = df["PHQ4"] - df.groupby("pid")["PHQ4"].transform("mean")

# 2. keep core + Elastic-Net picks, but USE PHQ4_dev instead of PHQ4
circadian_core = ["PHQ4_dev", "SRI", "dlmo_proxy", "sigma_bed", "mid_sleep"]
enet_vars      = pd.read_csv("data/processed/selected_predictors.csv")["PHQ4_dev"]

vars_all = list(dict.fromkeys(circadian_core + enet_vars.tolist()))

vars_all = [v for v in vars_all if v != "PHQ4_dev"] + ["PHQ4_dev"]
p        = len(vars_all)

# z-score features

In [9]:
def zscore_subject(group: pd.DataFrame) -> pd.DataFrame:
    z = group[vars_all].copy()
    # z-score every column **except** PHQ4_dev
    for col in vars_all:
        if col == "PHQ4_dev":
            continue                   # leave raw deviation
        mu  = z[col].mean()
        sd  = z[col].std(ddof=0)
        z[col] = (z[col] - mu) / (sd if sd > 0 else 1.0)
    return z.fillna(0)                 # residual NaN → 0

df_z = (
    df.groupby("pid", sort=False)
      .apply(zscore_subject)
      .reset_index(level=0)            # keep pid as a column
)

/var/folders/j_/3nq1pt5x6yqcx3k5d68vn4mc0000gq/T/ipykernel_83373/827302596.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(zscore_subject)


# Stack datasets

In [11]:
T = 10                      # longest window you want to model
pids = df_z["pid"].unique()
N = len(pids)

X_t = np.zeros((T, N, p), dtype=np.float64)

for subj_idx, pid in enumerate(pids):
    seq = df_z.loc[df_z["pid"] == pid, vars_all].to_numpy()
    t_len = min(len(seq), T)
    X_t[:t_len, subj_idx, :] = seq[:t_len]      # left-align (earliest first)

# Try Lingam

In [12]:
from lingam import LongitudinalLiNGAM
model = LongitudinalLiNGAM(n_lags=2, random_state=0)
model.fit(X_t)

/Users/kwang/.local/share/mamba/envs/lingam-env/lib/python3.10/site-packages/sklearn/linear_model/_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 15 iterations, i.e. alpha=3.152e-04, with an active set of 15 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/Users/kwang/.local/share/mamba/envs/lingam-env/lib/python3.10/site-packages/sklearn/linear_model/_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 16 iterations, i.e. alpha=1.385e-05, with an active set of 16 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/Users/kwang/.local/share/mamba/envs/lingam-env/lib/python3.10/site-packages/sklearn/linear_model/_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 17 i

# View graph and causal effects estimation

In [14]:
# -----------------------------------------------------------
# Visualise and inspect LongitudinalLiNGAM results
# -----------------------------------------------------------
from lingam.utils import make_dot
import graphviz, numpy as np, pandas as pd

# -- helper --------------------------------------------------
def edge_table(B, src_lab, tgt_lab, lag, thr=0.05):
    i, j = np.where(np.abs(B) > thr)
    return pd.DataFrame({
        "lag"   : lag,
        "from"  : [src_lab[a] for a in i],
        "to"    : [tgt_lab[b] for b in j],
        "weight": B[i, j].round(3)
    })

# -----------------------------------------------------------
# 1.  Instantaneous graph  B(t,t)   (last time-point)
# -----------------------------------------------------------
t_last   = X_t.shape[0] - 1
p        = len(vars_all)
labels_t = [f"{v}(t)" for v in vars_all]

B_inst   = model.adjacency_matrices_[t_last, 0]
dot_inst = make_dot(B_inst, labels=labels_t)
dot_inst.render("inst_graph", format="png", view=True)   # saves & opens PNG

# -----------------------------------------------------------
# 2.  Lag-1 graph  B(t,t-1)
#     (place edges in lower-left quadrant of a 2p×2p matrix)
# -----------------------------------------------------------
B_lag1   = model.adjacency_matrices_[t_last, 1]
B_big    = np.zeros((2*p, 2*p))
B_big[p:, :p] = B_lag1

labels_lag = [f"{v}(t-1)" for v in vars_all] + [f"{v}(t)" for v in vars_all]
dot_lag1   = make_dot(B_big, labels=labels_lag)
dot_lag1.render("lag1_graph", format="png", view=True)

# -----------------------------------------------------------
# 3.  Edge list (threshold |w| ≥ 0.05)
# -----------------------------------------------------------
edges0 = edge_table(B_inst, vars_all, vars_all, lag=0)
edges1 = edge_table(B_lag1,
                    [f"{v}(t-1)" for v in vars_all],
                    [f"{v}(t)"   for v in vars_all],
                    lag=1)
edges  = pd.concat([edges0, edges1], ignore_index=True)
print("\n=== Direct edges (|w| ≥ 0.05) ===")
print(edges.sort_values("weight", key=np.abs, ascending=False).to_string(index=False))

# -----------------------------------------------------------
# 4.  Quick total effect  SRI → PHQ4_dev  (same-time)
# -----------------------------------------------------------
from lingam import DirectLiNGAM
X_flat = X_t.reshape(-1, p)          # treat each (subject,time) row as i.i.d.
dlin   = DirectLiNGAM(random_state=0).fit(X_flat)

src = vars_all.index("SRI")
tgt = vars_all.index("PHQ4_dev")
tot = dlin.estimate_total_effect(X_flat, src, tgt)
print(f"\nTotal effect  SRI → PHQ4_dev  = {tot:.3f}")


=== Direct edges (|w| ≥ 0.05) ===
 lag                                                                                   from                                                                                   to  weight
   0       slp_fitbit_sleep_intraday_rapids_ratiodurationawakeunifiedwithinmain_evening_std    slp_fitbit_sleep_intraday_rapids_ratiodurationasleepunifiedwithinmain_evening_std   1.000
   0        slp_fitbit_sleep_intraday_rapids_ratiodurationasleepunifiedwithinmain_night_std       slp_fitbit_sleep_intraday_rapids_ratiodurationawakeunifiedwithinmain_night_std   1.000
   1                                                                          PHQ4_dev(t-1)    slp_fitbit_sleep_intraday_rapids_ratiodurationawakeunifiedwithinmain_night_std(t)   0.868
   1                                                                          PHQ4_dev(t-1)   slp_fitbit_sleep_intraday_rapids_ratiodurationasleepunifiedwithinmain_night_std(t)  -0.831
   1              steps_fitbit_steps_int

/Users/kwang/.local/share/mamba/envs/lingam-env/lib/python3.10/site-packages/lingam/base.py:76: UserWarning: The estimated causal effect may be incorrect because the causal order of the destination variable (to_index=23) is earlier than the source variable (from_index=0).
  warnings.warn(


In [19]:
import numpy as np
import pandas as pd
from lingam import DirectLiNGAM

# 1️⃣ Identify direct parents of PHQ4_dev at t = last
t_last = X_t.shape[0] - 1
p      = len(vars_all)
target = vars_all.index("PHQ4_dev")
thr    = 0.05

parents = []
# model.adjacency_matrices_[t, lag] has shape (p,p)
for lag in range(model.adjacency_matrices_.shape[1]):
    B = model.adjacency_matrices_[t_last, lag]
    idx = np.where(np.abs(B[target, :]) > thr)[0]
    for i in idx:
        parents.append((vars_all[i], lag))
# unique
parents = list(dict.fromkeys(parents))
print(len(parents))
print("Direct parents of PHQ4_dev (var, lag):")
for parent in parents:
    print(f"  {parent[0]} (lag {parent[1]})")

29
Direct parents of PHQ4_dev (var, lag):
  slp_fitbit_sleep_intraday_rapids_mindurationawakeunifiedmain_afternoon_slope (lag 0)
  slp_fitbit_sleep_summary_rapids_countepisodemain_allday_mean (lag 0)
  slp_fitbit_sleep_intraday_rapids_stddurationasleepunifiedmain_afternoon_mean (lag 0)
  SRI (lag 1)
  dlmo_proxy (lag 1)
  sigma_bed (lag 1)
  slp_fitbit_sleep_intraday_rapids_mindurationawakeunifiedmain_afternoon_slope (lag 1)
  steps_fitbit_steps_intraday_rapids_mindurationsedentarybout_allday_std (lag 1)
  steps_fitbit_steps_intraday_rapids_mindurationactivebout_night_mean (lag 1)
  slp_fitbit_sleep_intraday_rapids_ratiodurationasleepunifiedwithinmain_night_std (lag 1)
  slp_fitbit_sleep_intraday_rapids_ratiodurationawakeunifiedwithinmain_night_std (lag 1)
  steps_fitbit_steps_intraday_rapids_avgdurationactivebout_night_slope (lag 1)
  screen_phone_screen_rapids_firstuseafter00unlock_locmap_home_afternoon_std (lag 1)
  steps_fitbit_steps_intraday_rapids_sumdurationactivebout_evening_sl

In [21]:
# 2️⃣ Static total‐effect estimates via DirectLiNGAM
X_flat = X_t.reshape(-1, p)                # (T·N, p)
dlin    = DirectLiNGAM(random_state=0).fit(X_flat)

static_effects = []
for var, lag in parents:
    src = vars_all.index(var)
    tot = dlin.estimate_total_effect(X_flat, src, target)
    static_effects.append({
        "variable": var,
        "lag": lag,
        "total_effect": round(tot, 3)
    })

df_static = pd.DataFrame(static_effects)
print("\n=== Static Total Effects (all history collapsed) ===")
display(df_static)


=== Static Total Effects (all history collapsed) ===


/Users/kwang/.local/share/mamba/envs/lingam-env/lib/python3.10/site-packages/lingam/base.py:76: UserWarning: The estimated causal effect may be incorrect because the causal order of the destination variable (to_index=23) is earlier than the source variable (from_index=5).
  warnings.warn(
/Users/kwang/.local/share/mamba/envs/lingam-env/lib/python3.10/site-packages/lingam/base.py:76: UserWarning: The estimated causal effect may be incorrect because the causal order of the destination variable (to_index=23) is earlier than the source variable (from_index=20).
  warnings.warn(
/Users/kwang/.local/share/mamba/envs/lingam-env/lib/python3.10/site-packages/lingam/base.py:76: UserWarning: The estimated causal effect may be incorrect because the causal order of the destination variable (to_index=23) is earlier than the source variable (from_index=0).
  warnings.warn(
/Users/kwang/.local/share/mamba/envs/lingam-env/lib/python3.10/site-packages/lingam/base.py:76: UserWarning: The estimated causal

,variable,lag,total_effect
0,slp_fitbit_sleep_intraday_rapids_mindurationaw...,0,-0.047
1,slp_fitbit_sleep_summary_rapids_countepisodema...,0,-0.084
2,slp_fitbit_sleep_intraday_rapids_stddurationas...,0,0.079
3,SRI,1,-0.020
4,dlmo_proxy,1,-0.015
5,sigma_bed,1,-0.007
6,slp_fitbit_sleep_intraday_rapids_mindurationaw...,1,-0.047
7,steps_fitbit_steps_intraday_rapids_minduration...,1,0.045
8,steps_fitbit_steps_intraday_rapids_minduration...,1,0.072
9,slp_fitbit_sleep_intraday_rapids_ratioduration...,1,-0.176


The code below takes way too long.

In [18]:
import warnings
from tqdm.notebook import tqdm
warnings.simplefilter("ignore", FutureWarning)
# 3️⃣ (Optional) Dynamic total‐effect from each lagged parent → PHQ4_dev(t)
dyn_eff = []
for var, lag in tqdm(parents):
    src = vars_all.index(var)
    try:
        eff_dyn = model.estimate_total_effect2(
            from_t    = lag,
            from_index= src,
            to_t      = t_last,
            to_index  = target
        )
    except ValueError:
        # no valid causal order for t=0 slices, skip
        continue
    dyn_eff.append({
        "variable": var,
        "from_time": f"t-{lag}",
        "to_time": f"t",
        "dynamic_total_effect": round(eff_dyn, 3)
    })

df_dyn = pd.DataFrame(dyn_eff)
print("\n=== Dynamic Total Effects (per lag) ===")
display(df_dyn)

  0%|          | 0/29 [00:00<?, ?it/s]

KeyboardInterrupt: 

Individual estimation is very difficult.

Below is a recipe for “per‐individual” causal effects, in two senses:

Dynamic direct effects by subject & time (weights are the same across subjects, but you can record them per‐subject/time)

True subject‐specific effects by refitting a separate LiNGAM on each person (only possible if you have enough timepoints per subject), but we don't have enough timepoints per subject.